In [1]:
import hail as hl

hl.stop()
hl.init()

from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

2022-09-28 10:42:48 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://compa021.hpc.in.bmrc.ox.ac.uk:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /gpfs3/users/lindgren/hjo721/hail/hail-20220928-1042-0.2.99-57537fea08d4.log


Loading BokehJS ...

In [2]:
mt = hl.read_matrix_table('/well/lindgren/barney/outofafr.mt')

In [3]:
mt = mt.annotate_cols(y = hl.rand_norm(0, 1))
mt = mt.annotate_cols(PRS = hl.rand_norm(0, 1))

stats = hl.agg.stats(mt.PRS)

mean = mt.aggregate_cols(stats.mean)
stdev = mt.aggregate_cols(stats.stdev)

mt = mt.annotate_cols(PRS_norm = (mt.PRS - mean)/stdev)

In [4]:
mt.col.show()

,,,
s,y,PRS,PRS_norm
str,float64,float64,float64
"""tsk_0""",-1.13e+00,-3.52e-01,-3.72e-01
"""tsk_1""",1.45e+00,5.24e-01,5.09e-01
"""tsk_2""",8.49e-01,1.88e+00,1.87e+00
"""tsk_3""",1.55e+00,5.92e-01,5.77e-01
"""tsk_4""",-9.60e-02,-3.54e-01,-3.74e-01
"""tsk_5""",-9.17e-01,9.91e-01,9.77e-01
"""tsk_6""",3.14e-01,-6.72e-01,-6.93e-01
"""tsk_7""",2.44e+00,-6.80e-01,-7.01e-01


In [5]:
mt.aggregate_cols(hl.agg.stats(mt.PRS_norm))

Struct(mean=7.105427357601002e-19, stdev=1.0000000000000002, min=-3.6056087606533924, max=4.3437838019710675, n=10000, sum=7.105427357601002e-15)

In [6]:
mt = hl.experimental.ldscsim.normalize_genotypes(mt.GT)

In [7]:
mt = mt.annotate_entries(mult=mt.norm_gt * mt.PRS)

In [20]:
mt.mult.show()

,,,,
,,'tsk_0','tsk_1','tsk_2'
locus,alleles,mult,mult,mult
locus<GRCh38>,array<str>,float64,float64,float64
chr2:34,"[""A"",""G""]",3.52e-03,-5.24e-03,-1.88e-02
chr2:100,"[""T"",""A""]",6.11e-03,-9.08e-03,-3.25e-02
chr2:167,"[""C"",""A""]",3.52e-03,-5.24e-03,-1.88e-02
chr2:182,"[""T"",""C""]",2.40e-02,-3.56e-02,-1.28e-01
chr2:210,"[""C"",""T""]",8.64e-03,-1.28e-02,-4.60e-02
chr2:219,"[""G"",""C""]",7.05e-03,-1.05e-02,-3.75e-02
chr2:299,"[""G"",""C""]",3.52e-03,-5.24e-03,-1.88e-02


In [8]:
gwas = hl.linear_regression_rows(y=mt.y,
                                 x=mt.mult,
                                 covariates=[1.0])
gwas.row.describe()

2022-09-28 10:43:05 Hail: INFO: linear_regression_rows: running on 10000 samples for 1 response variable y,
    with input variable x, and 1 additional covariate...


--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh38>, 
        alleles: array<str>, 
        n: int32, 
        sum_x: float64, 
        y_transpose_x: float64, 
        beta: float64, 
        standard_error: float64, 
        t_stat: float64, 
        p_value: float64
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


In [9]:
p = hl.plot.qq(gwas.p_value)
show(p)

2022-09-28 10:43:13 Hail: INFO: Ordering unsorted dataset with network shuffle1]
